In [1]:
import requests
import psycopg2
import pandas as pd
from datetime import datetime
import pytz

In [2]:
apiKey = input('Enter API key: ')

In [3]:
sport = 'basketball_nba'
region = 'us,us2'
markets = 'h2h'
oddsFormat = 'american'

In [4]:
response = requests.get(f'https://api.the-odds-api.com/v4/sports/{sport}/odds?apiKey={apiKey}&regions={region}&markets={markets}&dateFormat=iso&oddsFormat={oddsFormat}&includeLinks=false&includeSids=true&includeBetLimits=true')
print(response)

<Response [200]>


In [5]:
data = response.json()

In [6]:

conn = psycopg2.connect(
    dbname="max_griffin_project",
    user="local",
    password="local_user123",
    host="localhost", 
    port="5432"
)
cur = conn.cursor()

try:
    for i in range(len(data)):
        event_id = data[i]["id"]
        sport = data[i]["sport_key"]
        home_team = data[i]["home_team"]
        away_team = data[i]["away_team"]
        start_time = datetime.strptime(data[i]["commence_time"], "%Y-%m-%dT%H:%M:%SZ")
        start_time = start_time.replace(tzinfo=pytz.UTC)

        cur.execute("INSERT INTO event (event_id, sport, home_team, away_team, start_time) VALUES (%s, %s, %s, %s, %s)", (event_id, sport, home_team, away_team, start_time))
        for j in range(len(data[i]["bookmakers"])):
            bookie_id = data[i]["bookmakers"][j]["key"]
            bookie_name = data[i]["bookmakers"][j]["title"]
            cur.execute("INSERT INTO bookmaker (bookmaker_id, name) VALUES (%s, %s) ON CONFLICT (bookmaker_id) DO NOTHING", (bookie_id, bookie_name))
            market_id = data[i]["bookmakers"][j]["markets"][0]["key"]
            cur.execute("INSERT INTO market (market_id, name) VALUES (%s, %s) ON CONFLICT (market_id) DO NOTHING", (market_id, "MoneyLine"))
            timestamp = datetime.strptime(data[i]["bookmakers"][j]["markets"][0]["last_update"], "%Y-%m-%dT%H:%M:%SZ")
            timestamp = timestamp.replace(tzinfo=pytz.UTC)
            conn.commit()
            for k in range(len(data[i]["bookmakers"][j]["markets"][0]["outcomes"])):
                team = data[i]["bookmakers"][j]["markets"][0]["outcomes"][k]["name"]
                price = f'{data[i]["bookmakers"][j]["markets"][0]["outcomes"][k]["price"]}'
                cur.execute("INSERT INTO liveline (liveline_id, price, team, timestamp, sport, bookmaker_id, event_id, market_id) VALUES (uuid_generate_v4(), %s, %s, %s, %s, %s, %s, %s)", (price, team, timestamp, sport, bookie_id, event_id, market_id))
    conn.commit()
except Exception as e:
    conn.rollback()
    print(f'Database Error: {e}')

cur.close()
conn.close()



In [7]:
print(data[0]["sport_key"])

basketball_nba
